In [1]:
import pandas as pd
import numpy as np
import glob 
import altair as alt
import sqlite3
#!pip install pandas-profiling
#!pip install https://github.com/ydataai/pandas-profiling/archive/master.zip
from pandas_profiling import ProfileReport

In [2]:
from pandas.core.reshape.merge import string
#Se inicia con la obtención de los datos para el presente mes de 2022-12-*
path_base ="http://api.tvmaze.com/schedule/web?date=2020-12-"
archivo= pd.read_json(path_base +"01")


#Para ejecutar en otro equipo cambiar la ruta de la siguiente abajo Ruta_JSO.
#RutaJSON=r"C:\Users\Alejandro Moncada\Documents\LuloTP"
RutaJSON=r"C:\Users\anliscen\Documents\LuloTP\LuloTP"

#array=np.empty(31, str)
#se usa el Zfill para agregar digito falante a los datos <10.
for ix in range(31):
  #pd.read_json
  df=pd.read_json(path_base +str(ix+1).zfill(2))
  df.to_json(RutaJSON+'\json'+'\TV_Maze'+str(ix+1).zfill(2)+".json")
  print(path_base +str(ix+1).zfill(2) + ' Export Complete')

http://api.tvmaze.com/schedule/web?date=2020-12-01 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-02 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-03 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-04 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-05 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-06 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-07 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-08 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-09 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-10 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-11 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-12 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-13 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-14 Export Complete
http://api.tvmaze.com/schedule/web?date=2020-12-15 Export Comp

In [4]:
rutas = glob.glob(RutaJSON+"\json"+"\TV_Maze*"+".json")
partes_glob = [pd.read_json(ruta) for ruta in rutas]
TV_Maze = pd.concat(partes_glob)

# 1. Se importan los datos del mes de diciembre 2020
Se realiza la importación de la data solicitada, en este caso dado que el API Rest no permitió para schedula una la función year ya que devolvia los valores por defecto (los shows emitidos en la fecha actual) se optó por realizar la importación de schedule día por día, es un proceso que se puede mejorar dado que toma aproximadamente 30 segundos recorrer el for y obtener la información, dentro del bucle son creados los archivos JSON obtenidos y lo que realicé fué unir todos estos archivos usando glob.glob para asegurarme que todos los archivos inicien en TV_Maze y finalicen con .json.

Nota: La función zfill(2) la uso para indicar que los valores que solo contengan 1 digito completen el segundo digito con un cero (0) a la izquierda dado que es requisito para la fecha en el formato ISO-8601  AAAA-MM-DD.

In [5]:
bn= pd.DataFrame(TV_Maze._embedded.values.tolist())['show']
TV_Maze_Show=pd.json_normalize(bn)

## 2. Generación de DataFrames 
En esta sección contamos con los dataframes a trabajar, (1) TV_Maze que corresponde a la información obtenida de Schedule que nos da un vistazo general de la información.  Y (2) TV_Maze_Show, que expande la columna _embedded que contiene toda la información del Show que considero mucho más útil dado que encontramos un mayor cantidad de variables categóricas que servirán mas adelante.

En este caso nuestros segundo dataset contiene ya filtrados los datos que corresponden a TV_Shows emitidos en el mes de Dic-2020 por lo que no es necesario realizar una cruce de tablas para cumplir con esta condición.

#TV_Maze.head(5)
#TV_Maze.to_excel('TV_Maze.xlsx')
#pd.drop(TV_Maze['id','airestamp','_links','_embedded','summary','image','url'])


## 3. Profiling de los DF
En este proceso se hará uso como se solicitó de archivos profiling en HTML dado que permiten una mayor versatilidad a la hora de realizar la visualizaciones. 

In [13]:
#TV_Maze_Show.describe()
profile = ProfileReport(TV_Maze, title='Modelo TV Maze Schedule', html={'style':{'full_witdth':True}})
profile.to_file(RutaJSON+'\profiling'+'\Reporte TV Maze Schedule'+".html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

##### En base al reporte de TV Maze Schedule
Se encuentra que las columnas rating,_links y _embedded. En este punto no son tan útiles las dos primeras porque no aportan información valiosa ya que tienen un JSON embebido pero la mayor parte de los valores en el caso de rating son valores None, adicionalmente esta columna no presenta el ID de los datos.

##### Image
Contiene imágenes de los dato, sin embargo no son relevantes para el análisis en este punto. Puede servir para otro tipo de análisis. Sin embargo dado que 66.6% de los datos es una variable que puede ser suceptible a ser eliminada.

#### _embedded 
Ya en este punto se extrajo la información, es el dataset más valiosos tanto por la cantidad de información como porque permite vincular los DF TV_Maze y TV_Maze shows por medio de realizar un JOIN por medio del ID. EN ESTE PUNTO SE PUEDE ELIMINAR 

#### _ratings
Aporta información pero la mayor parte de los datos tiene este campo vacio, puede ser un indicador de popularidad de la serie.



In [14]:
#TV_Maze_Show.describe()
profile2 = ProfileReport(TV_Maze_Show, title='Modelo TV Maze Shows', html={'style':{'full_witdth':True}})
profile2.to_file(RutaJSON+'\profiling'+'\Reporte TV Maze Shows'+".html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\anliscen\AppData\Roaming\Python\Python39\site-packages\pandas_profiling\model\correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

#### En base al reporte de datos de TV_Maze Shows
Se encuentra un DF rico en datos, con demasiadas columnas. 

In [ ]:
#Distribución de Fechas
plot = TV_Maze_Show['type'].value_counts().plot(kind='pie',autopct='%.2f',
                                           figsize=(4,6),
                                           title='Distribución por Tipo de programa') 
plot


TV_Maze['runtime'].median()

TV_Regular=TV_Maze['type'].str.contains(['regular']) TV_Maze['runtime'].median()

TV_Special=TV_Maze['type'].str.contains(['insignificant_special']) TV_Maze['runtime'].median()

TV_Special_Insignificant=TV_Maze['type'].str.contains(['significant_special']) TV_Maze['runtime'].median()

In [ ]:
AverageRuntime = pd.crosstab(index=TV_Maze['runtime'],
            columns=TV_Maze['type'])
AverageRuntime

In [ ]:
pd.value_counts(TV_Maze['image'])

In [ ]:
#TV_Maze=(TV_Maze['type'].str.contains('regular'))
Regular=TV_Maze.loc[TV_Maze['type'] == 'significant_special']
ISpecial=TV_Maze.loc[TV_Maze['type'] == 'insignificant_special']
Special=TV_Maze.loc[TV_Maze['type'] == 'regular']


###### 6.A. Average Value from data Regular, Significant_Special and Significant_Special on type

In [ ]:
Special['runtime'].mean()

In [ ]:
ISpecial['runtime'].mean()

In [ ]:
Special['runtime'].mean()

In [ ]:
SCFull = pd.read_json("https://api.tvmaze.com/schedule/full")

In [ ]:
TV_Maze.columns